In [1]:
#first run initial setup notebook
#then on each machine install ndn
#then run nfd-start
#then use this notebook

import json
import traceback
import os
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
    
fablib.show_config()


-----------------------------------  ---------------------------------------------
credmgr_host                         cm.fabric-testbed.net
orchestrator_host                    orchestrator.fabric-testbed.net
fabric_token                         /home/fabric/.tokens.json
project_id                           6ce270de-788d-4e07-8bae-3206860a6387
bastion_username                     dfulop_0000025834
bastion_key_filename                 /home/fabric/work/fabric_config/bastion_key
bastion_public_addr                  bastion-1.fabric-testbed.net
bastion_passphrase                   None
slice_public_key_file                /home/fabric/work/fabric_config/slice_key.pub
slice_private_key_file               /home/fabric/work/fabric_config/slice_key
fabric_slice_private_key_passphrase  None
fablib_log_file                      /tmp/fablib/fablib.log
fablib_log_level                     INFO
-----------------------------------  ---------------------------------------------


In [2]:
slice = fablib.get_slice("NDNSlice")
consumers = [
    {'name':'c1', 'net':'192.168.1.0', 'ip':'192.168.1.2', 'router_ip':'192.168.1.1'},
    {'name':'c2', 'net':'192.168.1.0', 'ip':'192.168.1.3', 'router_ip':'192.168.1.1'},
    {'name':'c3', 'net':'192.168.1.0', 'ip':'192.168.1.4', 'router_ip':'192.168.1.1'},
    {'name':'c4', 'net':'192.168.1.0', 'ip':'192.168.1.5', 'router_ip':'192.168.1.1'},
    {'name':'c5', 'net':'192.168.1.0', 'ip':'192.168.1.6', 'router_ip':'192.168.1.1'}]
routers = [
        {'name':'r1'}
]

producer = {'name':'p', 'net':'192.168.0.0', 'ip':'192.168.0.2', 'router_ip':'192.168.0.1'}

producer['node'] = slice.get_node(producer['name']);
producer['usr'] = str(producer['node'].get_username())
producer['man_ip'] = str(producer['node'].get_management_ip())
producer['if'] = producer['node'].get_interfaces()[0].get_os_interface()

for r in routers:
    r['node'] = slice.get_node(r['name'])
    r['ifs'] =r['node'].get_interfaces()
    r['man_ip'] = str(r['node'].get_management_ip())
    r['usr'] = str(r['node'].get_username())
    for interface in r['ifs']:
        if interface.get_name() == f"{r['name']}-{r['name']}_p_nic-p1":
            producer['router_if'] = interface
            producer['router_if_name'] = interface.get_os_interface()
            producer['router'] = r

    
for c in consumers:
    c['node'] = slice.get_node(c['name'])
    c['man_ip'] = str(c['node'].get_management_ip())
    c['if'] = str(c['node'].get_interfaces()[0].get_os_interface())
    c['usr'] = str(c['node'].get_username())
    c['router'] = routers[0]
    c['router_name']=c['router']['name']
    
    for interface in c['router']['ifs']:
        # print(interface.get_name())
        if interface.get_name() == f"{c['router_name']}-{c['router_name']}_{c['name']}_nic-p1" or interface.get_name() == f"{c['router_name']}-{c['router_name']}_c_nic-p1":
            c['router_if'] = interface 
            c['router_if_name'] = interface.get_os_interface()

print(f'{producer}')
print(f'{consumers}')
print(f'{routers}')

# print(client.execute("echo hello"))
# print(server.execute("echo hi"))


{'name': 'p', 'net': '192.168.0.0', 'ip': '192.168.0.2', 'router_ip': '192.168.0.1', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7f2eafbdd0d0>, 'usr': 'ubuntu', 'man_ip': '131.94.57.23', 'if': 'ens7', 'router_if': <fabrictestbed_extensions.fablib.interface.Interface object at 0x7f2ea86083a0>, 'router_if_name': 'ens7', 'router': {'name': 'r1', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7f2eac6f48e0>, 'ifs': [<fabrictestbed_extensions.fablib.interface.Interface object at 0x7f2ea860adc0>, <fabrictestbed_extensions.fablib.interface.Interface object at 0x7f2ea86083a0>], 'man_ip': '131.94.57.62', 'usr': 'ubuntu'}}
[{'name': 'c1', 'net': '192.168.1.0', 'ip': '192.168.1.2', 'router_ip': '192.168.1.1', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7f2ea86070a0>, 'man_ip': '131.94.57.32', 'if': 'ens7', 'usr': 'ubuntu', 'router': {'name': 'r1', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7f2eac6f48e0>, 'ifs': [<fabrictestb

In [3]:
producer['node'].upload_file("/home/fabric/work/NDNProject/video_producer.py", "/home/ubuntu/video_producer.py")
for c in consumers:
    c['node'].upload_file("/home/fabric/work/NDNProject/video_consumer.py", "/home/ubuntu/video_consumer.py")

In [4]:
#start producer applicatio
producer['node'].execute_thread(f"sudo python3 video_producer.py")

<Future at 0x7f2ea85cf3a0 state=running>

SET EXPERIMENT VARIABLES

In [72]:
from time import sleep

router_latency = 100 #10 50 100 ... 1?
num_consumers = 2 #1,2,3,4,5
run_number = 1

In [73]:
res = routers[0]['node'].execute(f"sudo tc qdisc del dev {producer['router_if_name']} root tbf rate {router_latency}kbit latency 50ms burst 1540")
print(res)
res = routers[0]['node'].execute(f"sudo tc qdisc add dev {producer['router_if_name']} root tbf rate {router_latency}kbit latency 50ms burst 1540")
print(res)
for i in range(num_consumers):
    c = consumers[i]
    node = c['node']
    c['future'] = node.execute_thread(f"sudo python3 video_consumer.py -l /home/ubuntu/ndnlogs/exp1_node{i}_latency{router_latency}_{num_consumers}consumers_{run_number}.csv")
    sleep(10)
# sleep(500)


('', '')
('', '')


In [74]:
for i in range(num_consumers):
    c = consumers[i]
    print(c['future'].result())
    sleep(1)
    node = c['node']
    node.download_file(f"/home/fabric/work/NDNProject/ndnlogs/exp1_node{i}_latency{router_latency}_{num_consumers}consumers_{run_number}.csv", f"/home/ubuntu/ndnlogs/exp1_node{i}_latency{router_latency}_{num_consumers}consumers_{run_number}.csv")

('20|120|4\n120|4\n4\n20 120 4\nretrieved segment 0 at bitrate 19\n0.0015571117401123047s | 0s played | 0 segments played | 0/120 buffered | seg 0 | rate 19 | 0.001552581787109375s latency | 0 buffers | 0s spent buffering \n120 segments left\nretrieved segment 1 at bitrate 18\n10.956703901290894s | 0s played | 0 segments played | 1/119 buffered | seg 1 | rate 18 | 10.955112218856812s latency | 0 buffers | 0s spent buffering \n119 segments left\nretrieved segment 2 at bitrate 14\n22.54967951774597s | 0s played | 0 segments played | 2/118 buffered | seg 2 | rate 14 | 11.592944383621216s latency | 1 buffers | 22.549681425094604s spent buffering \n118 segments left\nretrieved segment 3 at bitrate 10\n22.581453561782837s | 0.031771183013916016s played | 1 segments played | 2/117 buffered | seg 3 | rate 10 | 0.031751155853271484s latency | 1 buffers | 22.549681425094604s spent buffering \n117 segments left\nretrieved segment 4 at bitrate 10\n26.843313217163086s | 4.293630361557007s played | 

In [59]:
sleep(1)
res = routers[0]['node'].execute(f"sudo tc qdisc del dev {producer['router_if_name']} root tbf rate {router_latency}kbit latency 50ms burst 1540")
print(res)
res = routers[0]['node'].execute(f"sudo tc qdisc add dev {producer['router_if_name']} root tbf rate {router_latency}kbit latency 50ms burst 1540")
print(res)
for i in range(num_consumers):
    c = consumers[i]
    node = c['node']
    c['future2'] = node.execute_thread(f"sudo python2 ~/AStream/dist/client/dash_client.py -m http://192.168.0.2/media/BigBuckBunny/4sec/BigBuckBunny_4s.mpd -p 'basic' -n 120 -d ")
    sleep(10)
# sleep(500)


('', '')
('', '')


In [60]:
for i in range(num_consumers):
    c = consumers[i]
    (c['future2'].result())
    sleep(1)
    node = c['node']
    res = node.execute("ls ./ASTREAM_LOGS")
    split = res[0].split('\n')
    file = ''
    for s in split:
        if s.find('.csv') != -1:
            file = s
            break 
    print(file)
    # node.download_file(f"/home/fabric/work/ndnlogs/astream_log_exp1_node{i}_latency{router_latency}_{num_consumers}consumers_{run_number}.json", f"/home/ubuntu/ASTREAM_LOGS/{split[0]}")
    node.download_file(f"/home/fabric/work/NDNProject/ndnlogs/dash_buffer_log_exp1_node{i}_latency{router_latency}_{num_consumers}consumers_{run_number}.csv", f"/home/ubuntu/ASTREAM_LOGS/{file}")
    # node.download_file(f"/home/fabric/work/ndnlogs/dash_runtime_log_exp1_node{i}_latency{router_latency}_{num_consumers}consumers_{run_number}.log", f"/home/ubuntu/ASTREAM_LOGS/{split[2]}")


DASH_BUFFER_LOG_2022-12-03.20_15_15.csv


In [55]:
#clear ndn caches and remove astream logs
sleep(10)
try:
    for node in slice.get_nodes():
        node.execute("nfdc cs erase /")
        node.execute("sudo rm ./ASTREAM_LOGS/*")
except Exception as e:
    print(f"Exception: {e}")


In [ ]:
client.execute("sudo tc qdisc add dev ens7 root tbf rate 10kbit latency 50ms burst 1540")
client.execute("sudo tc qdisc add dev ens7 root tbf rate 50kbit latency 50ms burst 1540")
client.execute("sudo tc qdisc add dev ens7 root tbf rate 100kbit latency 50ms burst 1540")